<a href="https://colab.research.google.com/github/JSJeong-me/GPT-Web/blob/main/191-%20Llamaindex.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install llama-index html2text

In [2]:
import os
# from llama_index import VectorStoreIndex, SimpleWebPageReader
# Enter your OpenAI key below:
os.environ["OPENAI_API_KEY"] = "sk-"

In [3]:
from llama_index import VectorStoreIndex, download_loader

SimpleWebPageReader = download_loader("SimpleWebPageReader")

loader = SimpleWebPageReader()

In [4]:
# URL you want to load into your vector store here:
url = "http://www.paulgraham.com/fr.html"
# Load the URL into documents (multiple documents possible)
documents = SimpleWebPageReader(html_to_text=True).load_data([url])
# Create vector store from documents
index = VectorStoreIndex.from_documents(documents)
# Create query engine so we can ask it questions:
query_engine = index.as_query_engine()
# Ask as many questions as you want against the loaded data:
response = query_engine.query("What are the 3 best advise by Paul to raise money?")


In [5]:
print(response)

1. Have multiple plans depending on how much you can raise and tailor your pitch to match different investors.
2. Underestimate the amount you hope to raise initially to create a sense of urgency and show progress.
3. Aim to be profitable without raising additional money, as it puts you in a stronger position and makes you less desperate in the eyes of investors.


LangChain


In [6]:
!pip install openai
!pip install python-dotenv
!pip install langchain

In [1]:
!echo "OPENAI_API_KEY=sk-z" >> .env
!source /content/.env

In [2]:
import os
import openai
import sys

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file

openai.api_key  = os.environ['OPENAI_API_KEY']

In [3]:
from llama_index import VectorStoreIndex, download_loader
from langchain.agents import initialize_agent, Tool
from langchain.llms import OpenAI
from langchain.chains.conversation.memory import ConversationBufferMemory

SimpleWebPageReader = download_loader("SimpleWebPageReader")

In [4]:


loader = SimpleWebPageReader()
documents = loader.load_data(urls=['http://www.paulgraham.com/fr.html'])
index = VectorStoreIndex.from_documents(documents)

In [ ]:
index.as_query_engine().query("What are the 3 best advise by Paul to raise money?")

In [7]:
tools = [
    Tool(
        name="Website Index",
        func=lambda q: index.as_query_engine(),
        description=f"Useful when you want answer questions about the text on websites.",
    ),
]

In [10]:
llm = OpenAI(temperature=1)
memory = ConversationBufferMemory(memory_key="chat_history")
agent_chain = initialize_agent(
    tools, llm, agent="zero-shot-react-description", memory=memory
)

output = agent_chain.run(input="What are the 3 best advise by Paul to raise money?")

In [11]:
output

'The top 3 pieces of advice from Paul on raising money are: [insert top 3 advice here].'